In [1]:
import pandas as pd
import os
import sqlite3 as sql3

In [2]:
currentdir = os.path.dirname(os.path.abspath(''))
db_path = f"{currentdir}\\S30 ETL Assignment.db"

In [28]:
# Get table names
def get_tabes():
    conn = sql3.connect(db_path)
    with conn:
        cur = conn.cursor()
        cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
        results = cur.fetchall()
        table_names = [x[0] for x in results if 'sqlite_sequence' not in x[0]]
    
    return table_names

# Convert tables to DFs 
def table_to_df(table_name):
    conn = sql3.connect(db_path)
    with conn:
        cur = conn.cursor()

        # Fetch table body
        cur.execute(f"SELECT * FROM {table_name}")
        
        table_body = cur.fetchall()

        # Fetch table headers
        cur.execute(f'PRAGMA table_info({table_name})',)
        results = cur.fetchall()
        table_headers = [column[1] for column in results]

        # Transform to DF
        df = pd.DataFrame(table_body, columns=table_headers)

    return df

In [26]:
# Transofrm SQLite3 DB to a Python Dict of DFs
dfs = dict()
for table in get_tabes():
    dfs[table] = table_to_df(table)

In [44]:
# Filter customers by age and merge to sales
bool_age_min = dfs['customers']['age'] >= 18
bool_age_max = dfs['customers']['age'] <= 35
df_customers = dfs['customers'][bool_age_min & bool_age_max]
df_customer_sales = df_customers.merge(dfs['sales'],on='customer_id',how='inner')

In [63]:
# Merge orders to items, filter out NULL quantities, and remove decimals points (without rounding)
df_orders_items = dfs['orders'].merge(dfs['items'],on='item_id',how='inner')[['sales_id','quantity','item_name']]
bool_quantity_nulls = df_orders_items['quantity'] > 0
df_orders_items = df_orders_items[bool_quantity_nulls]
df_orders_items['quantity'] = df_orders_items['quantity'].astype('int64')

In [68]:
# Merge all, aggeragte and sort
df_merged_all = df_customer_sales.merge(df_orders_items, on='sales_id', how='inner')[['customer_id','age','item_name','quantity']]
df_agg = df_merged_all.groupby(['customer_id', 'age', 'item_name'])['quantity'].sum().reset_index()
df_final = df_agg.sort_values(by=['customer_id', 'quantity'], ascending=[True, False])

In [80]:
df_final.to_csv('test_pandas.csv',sep=';',index=False)